In [1]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
from google.colab import output

!mkdir "./train_imgs"
!unzip "/data/train_imgs.zip" -d "./train_imgs"

!mkdir "./test_imgs"
!unzip "/data/test_imgs.zip" -d "./test_imgs"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ./train_imgs/146-1-1-06-Z36_C-0000019.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_C-0000021.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_C-0000023.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_C-0000025.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_C-0000027.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_C-0000029.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_C-0000031.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000001.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000003.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000005.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000007.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000009.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000011.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000013.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000015.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000017.jpg  
  inflating: ./train_imgs/146-1-1-06-Z36_D-0000019.j

In [3]:
import pandas as pd
import numpy as np
import random
import cv2
import glob
import random
import warnings    
import matplotlib.pyplot as plt       
from keras_preprocessing.image import ImageDataGenerator
warnings.filterwarnings('ignore')       
%matplotlib inline

In [4]:
train_y = pd.read_csv("./data/train_augumentation.csv")
valid_y = pd.read_csv("./data/valid.csv")
precomputed_train = pd.read_csv("./data/precomputed_train.csv")
precomputed_valid = pd.read_csv("./data/precomputed_valid.csv")
columns = train_y.columns

In [5]:
datagen=ImageDataGenerator(rescale = 1/255) 
test_datagen=ImageDataGenerator(rescale = 1/255) 

train_generator=datagen.flow_from_dataframe(
    dataframe=train_y[:3776],
        directory='train_imgs/',
            x_col="image",
                y_col=columns[1:],
                    batch_size=1,
                        shuffle=False,
                            class_mode="raw",
                                target_size=(1080,1920))
valid_generator=test_datagen.flow_from_dataframe(
    dataframe=valid_y[:],
        directory='train_imgs/',
            x_col="image",
                y_col=columns[1:],
                    batch_size=1,
                        shuffle=False,
                            class_mode="raw",
                                target_size=(1080,1920))

Found 3776 validated image filenames.
Found 419 validated image filenames.


In [6]:
train_y = train_y.values[:,1:].astype(float)
valid_y = valid_y.values[:,1:].astype(float)

In [7]:
precomputed_train = precomputed_train.values[:,1:].astype(float)
precomputed_valid = precomputed_valid.values[:,1:].astype(float)
precomputed_train.shape, precomputed_valid.shape , train_y.shape, valid_y.shape

((36270, 2048), (419, 2048), (36270, 48), (419, 48))

In [8]:
import tensorflow as tf
my_rnet = tf.keras.applications.ResNet152V2(
    include_top=False, weights='imagenet',input_shape=(1080,1920,3), pooling = "avg")
my_rnet.trainable = False

234553344/234545216 [==============================] - 2s 0us/step


In [9]:
my_rnet.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
my_rnet.summary()

Model: "resnet152v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1080, 1920,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 1086, 1926, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 540, 960, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 542, 962, 64) 0           conv1_conv[0][0]                 
________________________________________________________________________________________

In [10]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor

In [ ]:
XGBR_best = XGBRegressor(max_depth= 10,
                         learning_rate = 0.01,
                         n_estimators = 1000,
                         tree_method='gpu_hist', 
                         gpu_id=0)
                         
Multi_XGBoost = MultiOutputRegressor(XGBR_best, n_jobs=-1)
Multi_XGBoost.fit(precomputed_train, train_y)                                             # 최적화된 하이퍼파라미터를 입력하고 훈련

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
Y_pred = Multi_XGBoost.predict(precomputed_valid)

In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/OpenPose/sample_submission.csv")
columns = test_df.columns

In [ ]:
test_datagen=ImageDataGenerator(rescale = 1/255)

test_generator = test_datagen.flow_from_dataframe(
     dataframe=test_df[:],
         directory='test_imgs',
             x_col="image",
                 y_col=columns[0],
                     batch_size=1,
                         shuffle=False,
                             class_mode="raw",
                                 target_size=(1080,1920))

In [ ]:
precomputed_test = my_rnet.predict(test_generator, verbose = 1)
pred = Multi_XGBoost.predict(precomputed_test)

In [ ]:
test_df.iloc[:,1:] = pred
test_df.to_csv('keypoint_XGBoost.csv', index=False)